# **1. Import libraries and load dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import os
import re
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim

,Unnamed: 0,title,content,source,url
0,0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,“Chơi thân không có nghĩa”,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Có thể buồn chút ít”,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [ ]:
def text_normalize(text):
  text = text.strip()

  return text

df['content'] = df['content'].apply(lambda x: text_normalize(x))
df.head()

,Unnamed: 0,title,content,source,url
0,0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,3,“Chơi thân không có nghĩa”,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Có thể buồn chút ít”,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [ ]:
for idx, row in df.iterrows():
  print(f'{idx+1}.')
  print (row['content'])
  print ()
  if idx == 10:
    break

1.
Bạn xấu như chiếc bóng
Cứ bám riết theo anh
Khi anh sáng, rực rỡ
Như mặt trời long lanh

Nhưng họ sẽ biến mất
Khi trời phủ mây đen
Tức là khi anh đói
Trong túi không có tiền

2.
Cái làm ta hạnh phúc
Thực ra cũng chẳng nhiều
Chỉ cần có ai đó
Để ta thầm thương yêu

Rồi thêm chút công việc
Cho ta làm hàng ngày
Cuối cùng, chút mơ mộng
Để đưa ta lên mây

3.
Chiều vừa xốp trên tay
Chợt nghe thoáng ong bay
Có ai vừa chết nhỉ
Mây thắt tang trăng gầy

Ớt đỏ sao cứ đỏ
Táo chín cho thật vàng
Em đẹp cho thêm đẹp
Để mắt ngừng lang thang

4.
Chơi thân không có nghĩa
Không cãi nhau bao giờ
Không làm nhau tức giận
Hay đôi lúc hững hờ

Vấn đề là ở chỗ
Đã chơi thân với nhau
Thì phải nên kiềm chế
Đừng giận dỗi quá lâu

5.
Có thể buồn chút ít
Một mình, không người yêu
Nhưng tình yêu bất hạnh
Thực sự buồn hơn nhiều

6.
Có thể có ông bố
Không yêu con; xưa nay
Đã ông là yêu cháu
Tôi khẳng định điều này

7.
Dưới giàn hoa thiên lý
Một mình anh đang ngồi
Không dưng em bước tới
Anh bỗng thành song đôi

8.
Đàn

# 2. **Pre-process data**

## 2.1. Tokenize data

In [ ]:
def tokenizer(text):
  return text.split()

def yield_tokens(df):
  for idx, row in df.iterrows():
    yield tokenizer(row['content'])

vocab = build_vocab_from_iterator(
    yield_tokens(df),
    specials=['<unk>', '<pad>', '<sos>', '<eos>', '<eol>'])
vocab.set_default_index(vocab['<unk>'])
vocab.get_stoi()

{'“Liệu': 1898,
 'ột': 1892,
 'ẩn': 1888,
 'ầm': 1887,
 'ảnh': 1885,
 'ướp': 1883,
 'ương': 1881,
 'Ước': 1878,
 'đồn': 1874,
 'đồi': 1873,
 'đẻ': 1868,
 'đấy!': 1861,
 'đại': 1859,
 'đò': 1852,
 'đìa': 1851,
 'đầu...': 1862,
 'đêm”': 1850,
 'đám': 1848,
 'đá': 1847,
 'đào': 1846,
 'đoạn': 1842,
 'đong': 1841,
 'đuốc': 1844,
 'điệu': 1840,
 'điều,': 1839,
 'điềm': 1838,
 'điêu': 1837,
 'điên': 1836,
 'đi,': 1835,
 'đao': 1833,
 'Đục': 1831,
 'Đợi': 1830,
 'Đỡ': 1829,
 'Đồi': 1827,
 'Đề': 1826,
 'Đến,': 1825,
 'Được': 1821,
 'Đơn': 1820,
 'Đông..."': 1819,
 'Đông': 1818,
 'Đây,': 1817,
 'Đàn': 1815,
 'Đà': 1814,
 'Đau': 1812,
 'úa': 1808,
 'ôm': 1807,
 'ô-kê': 1806,
 'ích': 1803,
 'âu': 1802,
 'à...': 1800,
 'Áng': 1795,
 'Ào': 1794,
 'xửa': 1793,
 'xốp': 1792,
 'xấu': 1790,
 'xăm': 1789,
 'xúm': 1788,
 'ướt': 1884,
 'xóm': 1785,
 'ếm': 1889,
 'xám': 1783,
 'xiết': 1777,
 'xuốn': 1782,
 'xem': 1774,
 'vực': 1770,
 'vỗ': 1767,
 'vỏ': 1765,
 'về.': 1762,
 'vả:': 1757,
 'vạn': 1756,
 'vượt

In [ ]:
PAD_TOKEN = vocab['<pad>']
SOS_TOKEN = vocab['<sos>']
EOS_TOKEN = vocab['<eos>']
EOL_TOKEN = vocab['<eol>']

MAX_SEQ_LEN = 25

def pad_and_truncate(input_ids, max_seq_len):
  if len(input_ids) > max_seq_len:
    input_ids = input_ids[:max_seq_len]
  else:
    input_ids += [PAD_TOKEN] * (max_seq_len - len(input_ids))
  return input_ids

def vectorize(text, max_seq_len):
  input_ids = [vocab[token] for token in tokenizer(text)]
  input_ids = pad_and_truncate(input_ids, max_seq_len)
  return input_ids

def decode (input_ids):
  return [vocab.get_itos()[idx] for idx in input_ids]

print (df['content'][0].split('\n')[0])
print (vectorize(df['content'][0].split('\n')[0], 10))

Bạn xấu như chiếc bóng
[392, 1790, 30, 316, 64, 1, 1, 1, 1, 1]


## 2.2. Prepare data for training

In [ ]:
class PoemDataset(Dataset):
    def __init__(self, df, tokenizer, vectorizer, max_seq_len):
        self.tokenizer = tokenizer
        self.vectorizer = vectorizer
        self.max_seq_len = max_seq_len
        self.input_seqs, self.target_seqs, self.padding_masks = self.create_samples(df)

    def create_padding_mask(self, input_ids, pad_token_id=PAD_TOKEN):
        return [0 if token_id == pad_token_id else 1 for token_id in input_ids]

    def split_content(self, content):
        samples = []

        poem_parts = content.split('\n\n')
        for poem_part in poem_parts:
            poem_in_lines = poem_part.split('\n')
            if len(poem_in_lines) == 4:
                samples.append(poem_in_lines)

        return samples

    def prepare_sample(self, sample):
        input_seqs = []
        target_seqs = []
        padding_masks = []

        input_text = '<sos> ' + ' <eol> '.join(sample) + ' <eol> <eos>'
        input_ids = self.tokenizer(input_text)
        for idx in range(1, len(input_ids)):
            input_seq = ' '.join(input_ids[:idx])
            target_seq = ' '.join(input_ids[1:idx+1])
            input_seq = self.vectorizer(input_seq, self.max_seq_len)
            target_seq = self.vectorizer(target_seq, self.max_seq_len)
            padding_mask = self.create_padding_mask(input_seq)

            input_seqs.append(input_seq)
            target_seqs.append(target_seq)
            padding_masks.append(padding_mask)

        return input_seqs, target_seqs, padding_masks

    def create_samples(self, df):
        input_seqs = []
        target_words = []
        padding_masks = []

        for idx, row in df.iterrows():
            content = row['content']
            samples = self.split_content(content)
            for sample in samples:
                sample_input_seqs, sample_target_words, sample_padding_masks = self.prepare_sample(sample)

                input_seqs += sample_input_seqs
                target_words += sample_target_words
                padding_masks += sample_padding_masks

        input_seqs = torch.tensor(input_seqs, dtype=torch.long)
        target_words = torch.tensor(target_words, dtype=torch.long)
        padding_masks = torch.tensor(padding_masks, dtype=torch.float)

        return input_seqs, target_words, padding_masks

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        input_seqs = self.input_seqs[idx]
        target_seqs = self.target_seqs[idx]
        padding_masks = self.padding_masks[idx]

        return input_seqs, target_seqs, padding_masks

In [ ]:
TRAIN_BS = 256
train_dataset = PoemDataset(
    df=df,
    tokenizer=tokenizer,
    vectorizer=vectorize,
    max_seq_len=MAX_SEQ_LEN
)
train_loader = DataLoader(
    train_dataset,
    batch_size=TRAIN_BS,
    shuffle=False
)

In [ ]:
input_seqs, target_seqs, padding_masks = next(iter(train_loader))

print(input_seqs[0])
print(target_seqs[0])
print(padding_masks[0])

tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1])
tensor([392,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])


In [ ]:
for idx in range(MAX_SEQ_LEN):
    print(decode(input_seqs[idx]))
    print(decode(target_seqs[idx]))

['<sos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['Bạn', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'Bạn', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['Bạn', 'xấu', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'Bạn', 'xấu', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

# **3. Model training**

## 3.1. Transformers architecture

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__ (self, embedding_dims, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dims, 2) * (-math.log(10000.0) / embedding_dims))
        pe = torch.zeros(max_len, 1, embedding_dims)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)

        return x

class TransformerModel(nn.Module):
  def __init__ (
      self,
      vocab_size,
      embedding_dims,
      num_heads,
      hidden_dims,
      num_layers,
      dropout=0.5
  ):
    super(TransformerModel, self).__init__()
    self.model_type = 'Transformer'
    self.embedding = nn.Embedding(vocab_size, embedding_dims)
    self.embedding_dims = embedding_dims

    self.pos_encoder = PositionalEncoding(embedding_dims, dropout)
    encoder_layers = nn.TransformerEncoderLayer(
        embedding_dims,
        num_heads,
        hidden_dims,
        dropout)

    self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
    self.linear = nn.Linear(embedding_dims, vocab_size)

    self.init_weights()

  def init_weights(self):
    initrange = 0.1
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.linear.bias.data.zero_()
    self.linear.weight.data.uniform_(-initrange, initrange)

  def forward(self, src, src_mask=None, padding_mask=None):
    src = self.embedding(src) * math.sqrt(self.embedding_dims)
    src = self.pos_encoder(src)
    if src_mask is None:
        src_mask = nn.Transformer.generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=padding_mask)
        output = self.linear(output)

        return output

In [ ]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMS = 128
HIDDEN_DIMS = 128
N_LAYERS = 2
N_HEADS = 4
DROPOUT = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_tests = torch.randint(1, 10, (1, 10)).to(device)

model = TransformerModel(
    VOCAB_SIZE,
    EMBEDDING_DIMS,
    N_HEADS,
    HIDDEN_DIMS,
    N_LAYERS,
    DROPOUT
).to(device)

with torch.no_grad():
    output = model(input_tests)
    print(output.shape)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


torch.Size([1, 10, 1902])


## 3.2. Training

In [ ]:
LR = 5.0
EPOCHS = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

In [ ]:
model.train()
for epoch in range(EPOCHS):
    losses = []
    for idx, samples in enumerate(train_loader):
        input_seqs, target_seqs, padding_masks = samples
        input_seqs = input_seqs.to(device)
        target_seqs = target_seqs.to(device)
        padding_masks = padding_masks.to(device).permute(1, 0)

        output = model(input_seqs, padding_mask=padding_masks)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target_seqs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f'EPOCH {epoch+1}\tLoss {total_loss}')
    scheduler.step()

EPOCH 1	Loss 5.234891777946835
EPOCH 2	Loss 4.345032612482707
EPOCH 3	Loss 3.5995196387881325
EPOCH 4	Loss 3.3401989142100015
EPOCH 5	Loss 3.1128567513965426
EPOCH 6	Loss 2.865499428340367
EPOCH 7	Loss 2.509659557115464
EPOCH 8	Loss 2.186702989396595
EPOCH 9	Loss 1.8539328234536308
EPOCH 10	Loss 1.5638934459005083
EPOCH 11	Loss 1.327026211080097
EPOCH 12	Loss 1.182730027607509
EPOCH 13	Loss 1.0149768193562825
EPOCH 14	Loss 0.9159795954113915
EPOCH 15	Loss 0.8289244288489932
EPOCH 16	Loss 0.7684967063722157
EPOCH 17	Loss 0.7081437962395805
EPOCH 18	Loss 0.6666047828538078
EPOCH 19	Loss 0.6242846477599371
EPOCH 20	Loss 0.5941629182724726
EPOCH 21	Loss 0.567550622281574
EPOCH 22	Loss 0.5580422949223292
EPOCH 23	Loss 0.5283481563840594
EPOCH 24	Loss 0.5213009388673873
EPOCH 25	Loss 0.48664534659612746
EPOCH 26	Loss 0.4701275967416309
EPOCH 27	Loss 0.46926532898630413
EPOCH 28	Loss 0.4565428864388239
EPOCH 29	Loss 0.4418307500226157
EPOCH 30	Loss 0.42390954352560495
EPOCH 31	Loss 0.41263130

In [ ]:
def sample_with_temperature(logits, temperature=1.0):
    if temperature != 1.0:
        logits = logits / temperature

    probabilities = F.softmax(logits, dim=-1)

    sampled_index = torch.multinomial(probabilities, 1).item()

    return sampled_index

In [ ]:
model.eval()
temperature = 1.2
input_text = '<sos> Anh'
input_tokens = tokenizer(input_text)
input_ids = [vocab[token] for token in input_tokens]
eos_token_id = vocab['<eos>']
generated_ids = input_ids.copy()
MAX_GENERATION_LEN = 50
for _ in range(MAX_GENERATION_LEN):
    input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)

    last_token_logits = outputs[0, -1, :]
    next_token_id = sample_with_temperature(last_token_logits, temperature)
    generated_ids.append(next_token_id)

    if next_token_id == eos_token_id:
        break

# Convert the generated tokens back to text
generated_text = decode(generated_ids)
generated_text = ' '.join(generated_text)
generated_text = generated_text.replace('<sos>', '')
lines = generated_text.split('<eol>')
for line in lines:
    print(''.join(line))

 Anh sống qua một ngọn núi vô cùng 
 Làng Hồ soi bảy cửa ngục 
 lão Bá Kiến 
 Suốt đời bạn đau xót hơn 
 vật nước 
 Người luôn miệng kêu bận 
 Long Quân tạm yên lòng 
 Quá chín trĩu cành sây 
 Đời qua một
